# wiki search

In [4]:
import requests
from bs4 import BeautifulSoup

In [41]:
# query = "SBL"
# url = ("https://zh.wikipedia.org/w/api.php?action=query&list=search&srsearch="+query+
#          "&format=json&formatversion=2")    
# response = requests.post(url)
# if response.status_code == requests.codes.ok:
#     result = response.json()
#     search_list = result["query"]["search"]
    
#     if len(search_list) > 0 and\
#         search_list[0].has_key("snippet") and\
#         search_list[0]["snippet"]:
#         snippet = search_list[0]["snippet"]
#         snippet = re.sub(r'</?\w+[^>]*>','',snippet)
#         print snippet

In [54]:
def wiki_query_list(query, lang):
    url = ("https://"+lang+".wikipedia.org/w/api.php?action=opensearch&search="+query+"&utf8")    
    response = requests.post(url)
    if response.status_code == requests.codes.ok:
        result = response.json()
    #     print result[0]
        if len(result[2]) == 0:
            return None
        
        content = result[2][0]
        if content == u"" or u"重定向" in content:  
            print 'redirect'
            return None
        
        return content

In [86]:
def wiki_query_snippet(query, lang):
    url = ("https://"+lang+".wikipedia.org/w/api.php?action=query&list=search&srsearch="+query+
             "&format=json&formatversion=2")    
    response = requests.post(url)
    if response.status_code == requests.codes.ok:
        result = response.json()
        search_list = result["query"]["search"]

        if len(search_list) > 0 and\
            search_list[0].has_key("snippet") and\
            search_list[0]["snippet"]:
            snippet = search_list[0]["snippet"]
            snippet = snippet.split(u'。')[0]
            snippet = re.sub(r'</?\w+[^>]*>','',snippet)+u'。'
            return snippet
    
    return None

In [60]:
# snippet = "臺北（臺灣話：Tâi-pak，俗體字書寫作台北）是臺灣的地理名詞，詞義為「臺灣為中心發展的都市。 「臺北"
# snippet_list = snippet.split('。')
# for i in snippet_list:
#     print i

In [84]:
def wiki_query(query):
    
    def check_contain_chinese(query):
        
        for ch in query:
            if u'\u4e00' <= ch <= u'\u9fff':
                return True
        return False
    
    if type(query) is not unicode:
        query = query.decode('utf-8')
    
    if check_contain_chinese(query):
        lang = 'zh'
    else:
        lang = 'en'
   
    for result in [wiki_query_list(query, lang), 
                   wiki_query_snippet(query, lang)]:
        if result:
            return result
    return None

In [81]:
print wiki_query('baseball')
print '---------------'
print wiki_query('cup')
print '---------------'
print wiki_query('蔡依林')
print '---------------'
print wiki_query('basketball')
print '---------------'
print wiki_query('林百里')
print '---------------'
print wiki_query('臺北')
print '---------------'
print wiki_query('六福村')
print '---------------'
print wiki_query('太陽的後裔')
print '---------------'
print wiki_query('柯文哲')

Baseball is a bat-and-ball game played between two opposing teams who take turns batting and fielding.
---------------
A cup is an open-top container used to hold liquids for pouring or drinking; it also can be used to store solids for pouring (e.g., sugar, flour, grains).
---------------
蔡依林（英语：Jolin Tsai，1980年9月15日－），出生於臺灣臺北縣新莊市（今新北市新莊區），是亞洲著名華語流行音樂女歌手。蔡依林因不斷革新其音樂風格與視覺形象而聞名，被視為推動舞曲成為大中華區主流流行音樂的重要人物，其如今完全主導於音樂事業上的方向與步驟，高製作水準的音樂作品更无可置疑。
---------------
Basketball is a team sport in which two teams, most commonly of five players each, opposing one another on a rectangular court, compete with the primary objective of shooting a basketball (approximately 9.4 inches (24 cm) in diameter) through the defender's hoop (a basket 18 inches (46 cm) in diameter mounted 10 feet (3.048 m) high to a backboard at each end of the court) while preventing the opposing team from shooting through their own hoop.
---------------
林百里（1949年4月24日－），是一位生於上海市的臺灣企業家，為廣達集團創辦人兼總裁、香港晶門科技前非執行主席。
---------------
臺北（臺灣

In [87]:
# 重定向問題
print wiki_query('籃球')
print '---------------'
print wiki_query('天龍八部')

redirect
籃球是一個由兩隊參與，在一個長方形籃球場進行的球類運動，每隊出場5名隊员，可將球向任何方向傳、投、拍、滾或運，目的是將籃球投入對方球籃得分，並阻止對方獲得控球權或得分。
---------------
redirect
王、忉利天（三十三天）、阎魔天、兜率天等诸天王，和天龙八部、神兵、四十圣众、道场神、华严神众等神现身并獲信仰等事，在《法华经》的灵山会上，天龙、乾闼婆、阿修罗等八部神集会时，其他的大乘经典神众也随之出现，因此「華嚴聖眾」也稱作「天龍八部」。


In [105]:
def google_search(query):
    if type(query) is not unicode:
        query = query.decode('utf-8')
        
    # Google 搜尋 URL
    google_url = 'https://www.google.com.tw/search'

    # 查詢參數
    my_params = {'q': query, 'lr':'lang_zh-TW'}

    # 下載 Google 搜尋結果
    r = requests.get(google_url, params = my_params)

    # 確認是否下載成功
    if r.status_code == requests.codes.ok:
      # 以 BeautifulSoup 解析 HTML 原始碼
      soup = BeautifulSoup(r.text, 'html.parser')

      # 觀察 HTML 原始碼
      # print(soup.prettify())

      # 以 CSS 的選擇器來抓取 Google 的搜尋結果
      items = soup.select('div.g > h3.r > a[href^="/url"]')
      for i in items[:5]:
        # 標題
        print(u"標題：" + i.text)
        # 網址
        print(u"網址：" + i.get('href').lstrip('/url?q='))

In [106]:
google_search('學游泳')

標題：如何自己學游泳2014.06.08更新 - 米達想想 - 痞客邦
網址：http://win588stock.pixnet.net/blog/post/155758403-%25E6%25B8%25B8%25E6%25B3%25B3%25E4%25B9%259F%25E5%258F%25AF%25E4%25BB%25A5%25E8%2587%25AA%25E5%25B7%25B1%25E5%25AD%25B8!!!-%25E5%25A6%2582%25E4%25BD%2595%25E8%2587%25AA%25E5%25B7%25B1%25E5%25AD%25B8%25E6%25B8%25B8%25E6%25B3%25B32014.06.08&sa=U&ved=0ahUKEwjB0KiLjJXgAhUBOrwKHUp-DlEQFggWMAA&usg=AOvVaw3fqYytO_KjGbrAZR-dGqhd
標題：[求助] 想學游泳要怎麼開始比較好呢？ - iPhone4.TW
網址：https://iphone4.tw/forums/showthread.php%3Ft%3D92608&sa=U&ved=0ahUKEwjB0KiLjJXgAhUBOrwKHUp-DlEQFggdMAE&usg=AOvVaw0PvjpeG_PsnqyQ_fOT-QNz
標題：幾歲開始學游泳最好？12個學游泳的關鍵問答/學前兒-玩中學/親子 ...
網址：https://www.parenting.com.tw/article/5077041-%25E5%25B9%25BE%25E6%25AD%25B2%25E9%2596%258B%25E5%25A7%258B%25E5%25AD%25B8%25E6%25B8%25B8%25E6%25B3%25B3%25E6%259C%2580%25E5%25A5%25BD%25EF%25BC%259F12%25E5%2580%258B%25E5%25AD%25B8%25E6%25B8%25B8%25E6%25B3%25B3%25E7%259A%2584%25E9%2597%259C%25E9%258D%25B5%25E5%2595%258F%25E7%25AD%2594/&sa=U&ved=0ahUKEwjB0KiLjJXgAhUBOrwKHU